# 02d Paralelismo

El paralelismo es la capacidad de ejecutar varias tareas simultáneamente, aprovechando que la mayoría de ordenadores actuales tienen una arquitectura de varios núcleos. Hay dos modelos principales de paralelismo:

* **Paralelismo por procesos:** Hay varias copias del mismo programa (proceso) ejecutándose a la vez, de modo que si abres el Administrador de tareas, verás que `Python` aparece varias veces.
* **Paralelismo por "hilos" (threads):** Hay un único proceso, que tiene varios conjuntos de código (threads) ejecutándose a la vez.

En general, el paralelismo por procesos es mejor para situaciones que requieren mucho uso de CPU, como cálculos matemáticos, mientras que el paralelismo por threads es mejor cuando la ejecución está limitada por operaciones de lectura y escritura. Además, la implementación estándar de Python, CPython, limita el número de threads ejecutándose simultáneamente a 1 (GIL), por lo que el paralelismo por threads no ofrece absolutamente ninguna ventaja en tareas de CPU.

In [1]:
# Código para importar Temporizador desde utils.py, no es importante

import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from utils import Temporizador

## Paralelismo por hilos

El paralelismo por hilos está implementado por el módulo `threading` de la librería estándar. Veremos solamente los conceptos más básicos, ya que en general nos interesarán más las tareas limitadas por CPU.

In [2]:
import threading 
from time import sleep

def f(x):
    sleep(5)
    print(x)

t1 = threading.Thread(target=f, args=("Thread 1\n",))
t2 = threading.Thread(target=f, args=("Thread 2\n",))

with Temporizador() as temp:
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(f"Tiempo total: {temp.ver_tiempo():.4f} s")

Thread 1

Thread 2

Tiempo total: 5.0057 s


Cada thread se crea como un objeto `threading.Thread`, donde `target` indica la función que se va a ejecutar, y `args` es una tupla con sus argumentos. El método `start()` inicia la ejecución del thread, y `join()` espera a que acabe. En este caso, al no tratarse de una tarea que esté limitada por CPU, el tiempo de ejecución se ha visto reducido.

En cambio, veamos qué ocurre con un cálculo matemático, calculando el factorial de los números entre 1000 y 2000:

In [11]:
def factorial(x):
    f = 1
    i = x
    while i > 0:
        f *= i
        i -=1
    return f

def tarea(inicio, fin, lista):
    for x in range(inicio, fin):
        lista[x-1000] = factorial(x)

In [14]:
resultado1 = [0,]*1000

with Temporizador() as temp:
    tarea(1000, 2000, resultado1)

print(f"Tiempo en un único thread: {temp.ver_tiempo():.4f} s")

Tiempo en un único thread: 0.3721 s


In [15]:
resultado2 = [0,]*1000
t1 = threading.Thread(target=tarea, args=(1000, 1500, resultado2))
t2 = threading.Thread(target=tarea, args=(1500, 2000, resultado2))

with Temporizador() as temp:
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(f"Tiempo en dos threads: {temp.ver_tiempo():.4f} s")

Tiempo en dos threads: 0.4132 s


In [34]:
for i in range(1000):
    if resultado1[i] != resultado2[i]:
        print(f"Error en el elemento {i}")
        break

El tiempo de ejecución es ligeramente mayor usando dos threads que uno solo! Además, no es posible obtener el valor de un `return`, por eso hemos tenido que pasar la lista "por referencia" para almacenar los valores. En este caso sencillo, cada thread escribía en elementos distintos, y no había posibilidad de que intentarán competir entre ellos. Pero en general, esto es una posibilidad que se puede evitar si uno de los threads bloquea temporalmente (lock) la ejecución del resto cuando tiene que usar un recurso compartido.

Finalmente, veamos que ambos threads comparten el mismo proceso. Una forma de hacerlo es abriendo el administrador de tareas, pero también podemos hacerlo desde python: el comando `os.getpid()` devuelve un identificador que es único para cada proceso:

In [11]:
def pid(x):
    sleep(3)
    print(f"Thread {x}, PID: {os.getpid()}\n")

t1 = threading.Thread(target=pid, args=(1,))
t2 = threading.Thread(target=pid, args=(2,))

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Proceso general, PID: {os.getpid()}")

Thread 1, PID: 42927

Thread 2, PID: 42927

Proceso general, PID: 42927


## Paralelismo por procesos

El paralelismo por procesos se realiza mediante el módulo `multiprocessing` de la librería estándar. El funcionamiento básico es muy similar a `threading`:

In [3]:
import multiprocessing
from time import sleep

def f(x):
    sleep(5)
    print(x)

p1 = multiprocessing.Process(target=f, args=("Process 1",))
p2 = multiprocessing.Process(target=f, args=("Process 2",))

with Temporizador() as temp:
    p1.start()
    p2.start()
    p1.join()
    p2.join()

print(f"Tiempo total: {temp.ver_tiempo():.4f} s")

Process 1
Process 2
Tiempo total: 5.0129 s


Pero en esta ocasión sí que obtenemos un (pequeño) beneficio al ejecutar en paralelo una tarea que requiera cálculos:

In [22]:
def factorial(x):
    f = 1
    i = x
    while i > 0:
        f *= i
        i -=1
    return f

def tarea(inicio, fin):
    lista = [factorial(x) for x in range(inicio, fin)]

Hay que notar que los procesos son programas completamente independientes entre sí, por lo que no es posible que compartan memoria, es decir, no podemos pasarles una lista para que la escriban. Más adelante veremos cómo devolver valores.

In [23]:
with Temporizador() as temp:
    tarea(1000, 2000)

print(f"Tiempo en un único thread: {temp.ver_tiempo():.4f} s")

Tiempo en un único thread: 0.4096 s


In [24]:
t1 = multiprocessing.Process(target=tarea, args=(1000, 1500))
t2 = multiprocessing.Process(target=tarea, args=(1500, 2000))

with Temporizador() as temp:
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(f"Tiempo en dos procesos: {temp.ver_tiempo():.4f} s")

Tiempo en dos procesos: 0.2653 s


Cada proceso tiene su identificador propio, y distinto del proceso general:

In [9]:
def pid(x):
    sleep(3)
    print(f"Proceso {x}, PID: {os.getpid()}\n")

t1 = multiprocessing.Process(target=pid, args=(1,))
t2 = multiprocessing.Process(target=pid, args=(2,))

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Proceso general, PID: {os.getpid()}")

Proceso 1, PID: 50339

Proceso 2, PID: 50340

Proceso general, PID: 42927


### Pools

En vez de gestionar los procesos individualmente, `multiprocessing` permite crear un "fondo común" de procesos a los que ir asignando trabajos, mediante el objeto `Pool`. Los trabajos se pueden asignar mediante las versiones paralelizadas de `map` o `starmap`. Funcionan de un modo similar a sus versiones no-paralelizadas, excepto que aquí se devuelve la lista calculada en vez de un itaredor. Esta es la forma más cómoda de paralelizar un cálculo en el que los pasos sean independientes entre sí:

In [15]:
with Temporizador() as temp:
    with multiprocessing.Pool(processes=2) as pool:
        resultado3 = pool.map(factorial, range(1000, 2000))

print(f"Tiempo en dos procesos: {temp.ver_tiempo():.4f} s")

Tiempo en dos procesos: 0.2990 s


El objeto `Pool` debe ser cerrado cuando acabemos de usarlo para poder devolver el control al proceso general. Para asegurarnos de que se cierra aunque se produzca una excepción, usamos un gestor de contexto.

Esta forma suele ser más conveniente que `start()`-`join()`, ya que no hay que especificar cómo dividir los argumentos entre procesos, y porque la manera de guardar los resultados es más natural.

En el ejemplo anterior hemos creado un pool de 2 procesos, pero podemos usar tantos procesos como queramos. Lo óptimo es usar, como mucho, un número de procesos igual al número de núcleos del ordenador. Si se solicitan más procesos, algunos de ellos tendrán que esperar a que otros terminen, por lo que no se produce ninguna ganancia en el tiempo de ejecución. La función `cpu_count()` devuelve el número de núcleos:

In [17]:
procesos = multiprocessing.cpu_count()

with Temporizador() as temp:
    with multiprocessing.Pool(processes=procesos) as pool:
        resultado3 = pool.map(factorial, range(1000, 2000))

print(f"Tiempo en {procesos} procesos: {temp.ver_tiempo():.4f} s")

Tiempo en 12 procesos: 0.1521 s


In [18]:
procesos = 2*multiprocessing.cpu_count()

with Temporizador() as temp:
    with multiprocessing.Pool(processes=procesos) as pool:
        resultado3 = pool.map(factorial, range(1000, 2000))

print(f"Tiempo en {procesos} procesos: {temp.ver_tiempo():.4f} s")

Tiempo en 24 procesos: 0.2661 s


También podemos asignar tareas individuales para que sean realizadas por uno de los procesos, mediante `apply`. La ejecución del resto de procesos se bloquea temporalmente hasta que termine:

In [38]:
def f(x):
    sleep(1.5)
    print(x)

with Temporizador() as temp:
    with multiprocessing.Pool(processes=2) as pool:
        pool.apply(f, ("Tarea 1",))
        pool.apply(f, ("Tarea 2",))

print(f"Tiempo en dos procesos: {temp.ver_tiempo():.4f} s")

Tarea 1
Tarea 2
Tiempo en dos procesos: 3.0265 s


El método `apply_async()` aplica la función pero de modo asíncrono, como una tarea de fondo, sin bloquear el resto de procesos:

In [39]:
with Temporizador() as temp:
    with multiprocessing.Pool(processes=2) as pool:
        pool.apply_async(f, ("Tarea 1",))
        pool.apply(f, ("Tarea 2",))

print(f"Tiempo en dos procesos: {temp.ver_tiempo():.4f} s")

Tarea 2Tarea 1

Tiempo en dos procesos: 1.5279 s


El hecho de que `apply_async()` no bloquee también tiene un inconveniente: el objeto `Pool` se puede cerrar sin esperar a que el proceso termine, con lo cual no obtenemos el resultado de la función:

In [9]:
def f(x):
    sleep(0.2)
    print(f"{x}, {os.getpid()}\n")

with multiprocessing.Pool(2) as pool:
    pool.apply_async(f, (1,))
    pool.apply_async(f, (2,))

Una posibilidad es usar al final del bloque un trabajo que sí bloquee (`apply`, `map`, `starmap`), y otra posibilidad es simplemente esperar:

In [28]:
with multiprocessing.Pool(2) as pool:
    pool.apply_async(f, (1,))
    pool.apply_async(f, (2,))
    sleep(0.3)

2, 34882

1, 34881



Los procesos asíncronos además tienen un método que bloquea la ejecución hasta que terminen, convirtiéndolas en procesos síncronos

In [10]:
with multiprocessing.Pool(2) as pool:
    a1 = pool.apply_async(f, (1,))
    a2 = pool.apply_async(f, (2,))
    a1.wait()
    a2.wait()

2, 7204
1, 7203




Si una función devuelve resultados, el valor devuelto por `apply_async()` no es el resultado directamente, sino un objeto de tipo `ApplyResult`. para recuperar el resultado, hay que usar el método `.get()`:

In [6]:
def f(x):
    sleep(0.2)
    return x**2

with multiprocessing.Pool(2) as pool:
    v = pool.apply_async(f, (5,))
    sleep(0.4)

print(v)
print(v.get())


25


Usar `.get()` dentro del `Pool` es una acción que bloquea, por lo que el programa esperará a que termine el cálculo:

In [8]:
with multiprocessing.Pool(2) as pool:
    v = pool.apply_async(f, (5,))
    v_res = v.get()

print(v_res)

25


Sin embargo, si el `Pool` se ha cerrado antes de finalizar el proceso asíncrono, al hacer `.get()`intentará esperar hasta que acabe el proceso, generando un bucle infinito:

In [7]:
with multiprocessing.Pool(2) as pool:
    v = pool.apply_async(f, (5,))

print(v)
print(v.get())

KeyboardInterrupt: 

## Memoria compartida

El módulo `multiprocessing` contiene varias estructuras para que los procesos compartan datos. En todos los casos, la compartición se realiza en tres pasos:
* Primero, el proceso que quiere acceder a la memoria compartida detiene temporalmente el resto de procesos. Esto se hace para asegurarse de que ningún otro proceso "compite" por el mismo recurso.
* A continuación, el proceso accede a la memoria.
* Finalmente se reanudan el resto de procesos.

### Queue

La estructura compartida más simple es `Queue`. Se trata de una lista a la que los procesos añaden valores al final. Los elementos de `Queue`, por lo tanto, estarán en el orden en que se han escrito, que en general no coincide con el oreden en el que se crean los procesos.

In [34]:
def cuadrado(x, q):
    sleep(max((10-x)/100, 0))
    q.put(x**2)

q = multiprocessing.Queue()
procesos = [multiprocessing.Process(target=cuadrado, args=(i, q)) for i in range(20)]

for p in procesos:
    p.start()

for p in procesos:
    p.join()

Los elementos almacenados en `Queue` se pueden obtener (ya sea por el proceso general, o por algún proceso hijo) mediante el método `.get()`. Esto devuelve el primer elemento de `Queue` y lo elimina:

In [35]:
for i in range(20):
    print(q.get())

100
121
81
144
169
64
196
225
256
49
289
324
36
361
25
16
9
4
1
0


Si intentamos obtener un valor de un `Queue` que ya ha sido vaciado, creamos un bucle infinito:

In [36]:
q.get()

KeyboardInterrupt: 

Podemos comprobar si está vacía con el método `.empty()`:

In [37]:
q = multiprocessing.Queue()
procesos = [multiprocessing.Process(target=cuadrado, args=(i, q)) for i in range(20)]

for p in procesos:
    p.start()

for p in procesos:
    p.join()

lista = []
while True:
    lista.append(q.get())
    if q.empty():
        break

print(lista)

[100, 121, 144, 81, 169, 196, 225, 64, 256, 289, 324, 49, 361, 36, 25, 16, 9, 4, 1, 0]


### Value y Array

La estructura `Value` permite compartir un valor entre procesos. El valor debe pertenecer a uno de los tipos elementales de C, y se especifica con un string que se pasa como primer argumento a `Value()`:
* `b`: char (1 byte)
* `u`: Carácter unicode
* `i`: int (2 bytes)
* `l`: long (4 bytes)
* `q`: long long (8 bytes)
* `f`: float (4 bytes)
* `d`: double (8 bytes)
* `B`, `I`, `L` y `Q` son versiones unsigned de los correspondientes tipos.

El valor almacenado se puede leer y escribir a través de la propiedad `.value`:

In [41]:
def f(x, v):
    sleep(x/100)
    pid = os.getpid()
    print(f"Yo soy el proceso {pid}, y me ejecuto después de {v.value}")
    v.value = pid

v = multiprocessing.Value("i", os.getpid())
procesos = [multiprocessing.Process(target=f, args=(i, v)) for i in range(20)]

for p in procesos:
    p.start()

for p in procesos:
    p.join()

Yo soy el proceso 11809, y me ejecuto después de 5315
Yo soy el proceso 11812, y me ejecuto después de 11809
Yo soy el proceso 11815, y me ejecuto después de 11812
Yo soy el proceso 11816, y me ejecuto después de 11815
Yo soy el proceso 11817, y me ejecuto después de 11816
Yo soy el proceso 11820, y me ejecuto después de 11817
Yo soy el proceso 11823, y me ejecuto después de 11820
Yo soy el proceso 11824, y me ejecuto después de 11823
Yo soy el proceso 11825, y me ejecuto después de 11824
Yo soy el proceso 11826, y me ejecuto después de 11825
Yo soy el proceso 11831, y me ejecuto después de 11826
Yo soy el proceso 11832, y me ejecuto después de 11831
Yo soy el proceso 11833, y me ejecuto después de 11832
Yo soy el proceso 11836, y me ejecuto después de 11833
Yo soy el proceso 11839, y me ejecuto después de 11836
Yo soy el proceso 11840, y me ejecuto después de 11839
Yo soy el proceso 11841, y me ejecuto después de 11840
Yo soy el proceso 11842, y me ejecuto después de 11841
Yo soy el p

Las operaciones `+=`, `-=`, `*=`, `/=`, `%=` leen y escriben el valor de una variable, por lo que su uso con memoria compartida puede llevar a situaciones en las que otro proceso haya modificado el valor entre la fase de lectura y la de escritura. Para asegurarse de que eso no ocurre, hay que bloquear temporalmente el resto de procesos con un `lock`:

In [43]:
def f(x, v, c):
    sleep(x/100)
    pid = os.getpid()
    print(f"Yo soy el proceso {pid}, y me ejecuto después de {v.value}. Han terminado {c.value} procesos.")
    v.value = pid
    with c.get_lock():
        c.value += 1

v = multiprocessing.Value("i", os.getpid())
contador = multiprocessing.Value("i", 0)
procesos = [multiprocessing.Process(target=f, args=(i, v, contador)) for i in range(20)]

for p in procesos:
    p.start()

for p in procesos:
    p.join()

Yo soy el proceso 12710, y me ejecuto después de 5315. Han terminado 0 procesos.
Yo soy el proceso 12713, y me ejecuto después de 12710. Han terminado 1 procesos.
Yo soy el proceso 12716, y me ejecuto después de 12713. Han terminado 2 procesos.
Yo soy el proceso 12717, y me ejecuto después de 12716. Han terminado 3 procesos.
Yo soy el proceso 12718, y me ejecuto después de 12717. Han terminado 4 procesos.
Yo soy el proceso 12723, y me ejecuto después de 12718. Han terminado 5 procesos.
Yo soy el proceso 12724, y me ejecuto después de 12723. Han terminado 6 procesos.
Yo soy el proceso 12725, y me ejecuto después de 12724. Han terminado 7 procesos.
Yo soy el proceso 12728, y me ejecuto después de 12725. Han terminado 8 procesos.
Yo soy el proceso 12731, y me ejecuto después de 12728. Han terminado 9 procesos.
Yo soy el proceso 12732, y me ejecuto después de 12731. Han terminado 10 procesos.
Yo soy el proceso 12733, y me ejecuto después de 12732. Han terminado 11 procesos.
Yo soy el proce

La estructura `Array` se usa para crear vectores de datos compartidos. Un `Array` tiene longitud fija, y todos los datos deben ser del mismo tipo, uno de los tipos elementales de C.

In [55]:
def f(x, a):
    sleep(x/100)
    pid = os.getpid()
    if x > 0:
        print(f"Soy el proceso {x} con PID {pid}. El proceso {x-1} tenía PID {a[x-1]}")
    a[x] = pid

a = multiprocessing.Array('i', 20)

procesos = [multiprocessing.Process(target=f, args=(i, a)) for i in range(20)]

for p in procesos:
    p.start()

for p in procesos:
    p.join()

Soy el proceso 1 con PID 14500. El proceso 0 tenía PID 14499
Soy el proceso 2 con PID 14501. El proceso 1 tenía PID 14500
Soy el proceso 3 con PID 14502. El proceso 2 tenía PID 14501
Soy el proceso 4 con PID 14503. El proceso 3 tenía PID 14502
Soy el proceso 5 con PID 14504. El proceso 4 tenía PID 14503
Soy el proceso 6 con PID 14507. El proceso 5 tenía PID 14504
Soy el proceso 7 con PID 14510. El proceso 6 tenía PID 14507
Soy el proceso 8 con PID 14511. El proceso 7 tenía PID 14510
Soy el proceso 9 con PID 14513. El proceso 8 tenía PID 14511
Soy el proceso 10 con PID 14517. El proceso 9 tenía PID 14513
Soy el proceso 11 con PID 14518. El proceso 10 tenía PID 14517
Soy el proceso 12 con PID 14519. El proceso 11 tenía PID 14518
Soy el proceso 13 con PID 14522. El proceso 12 tenía PID 14519
Soy el proceso 14 con PID 14525. El proceso 13 tenía PID 14522
Soy el proceso 15 con PID 14526. El proceso 14 tenía PID 14525
Soy el proceso 16 con PID 14527. El proceso 15 tenía PID 14526
Soy el proc

### Manager

Una estructura `Manager` permite compartir cualquier objeto de Python, aunque es más lento que `Value` y `Array`. Para crear listas y diccionarios compartidos, hay que usar los métodos `.list()` y `.dict()` respectivamente:

In [56]:
def f(x, l, d):
    sleep(x/100)
    pid = os.getpid()
    l.append(pid)
    d.update({x: pid})

with multiprocessing.Manager() as mgr:
    l1 = mgr.list()
    d1 = mgr.dict()

    procesos = [multiprocessing.Process(target=f, args=(i, l1, d1)) for i in range(20)]

    for p in procesos:
        p.start()

    for p in procesos:
        p.join()

    print(l1)
    print(d1)

[15466, 15472, 15475, 15478, 15481, 15486, 15487, 15495, 15497, 15502, 15505, 15508, 15514, 15517, 15520, 15523, 15526, 15532, 15535, 15540]
{0: 15466, 1: 15472, 2: 15475, 3: 15478, 4: 15481, 5: 15486, 6: 15487, 7: 15495, 8: 15497, 9: 15502, 10: 15505, 11: 15508, 12: 15514, 13: 15517, 14: 15520, 15: 15523, 16: 15526, 17: 15532, 18: 15535, 19: 15540}


Para compartir uno o varios `int`, `float` o `str` hay que introducirlos dentro de un `Namespace`. En este caso, para poder leer y escribir la variable, bloquearemos la ejecución con un `lock`:

In [97]:
def f(x, ns, lock):
    sleep(x/10)
    with lock:
        ns.texto += f"El proceso {x} tiene PID {os.getpid()}\n"

with multiprocessing.Manager() as mgr:
    ns = mgr.Namespace()
    lock = mgr.Lock()
    ns.texto = ""

    procesos = [multiprocessing.Process(target=f, args=(i, ns, lock)) for i in range(20)]

    for p in procesos:
        p.start()

    for p in procesos:
        p.join()

    print(ns.texto)

El proceso 0 tiene PID 21886
El proceso 1 tiene PID 21890
El proceso 2 tiene PID 21894
El proceso 3 tiene PID 21898
El proceso 4 tiene PID 21901
El proceso 5 tiene PID 21904
El proceso 6 tiene PID 21907
El proceso 7 tiene PID 21910
El proceso 8 tiene PID 21913
El proceso 9 tiene PID 21916
El proceso 10 tiene PID 21919
El proceso 11 tiene PID 21922
El proceso 12 tiene PID 21925
El proceso 13 tiene PID 21927
El proceso 14 tiene PID 21931
El proceso 15 tiene PID 21933
El proceso 16 tiene PID 21935
El proceso 17 tiene PID 21939
El proceso 18 tiene PID 21942
El proceso 19 tiene PID 21945



Para compartir objetos de otros tipos, incluyendo clases definidas por nosotros mismos, es un poco más complicado. Hay que crear nuestro propio `Manager` heredando desde la clase `BaseManager`, y registrar la clase para que la reconozca. En una instancia perteneciente a un `Manager`no es posible acceder directamente a los miembros, sino que hay que hacerlo a través de métodos:

In [98]:
import multiprocessing.managers

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

    def ver(self):
        return str(self)

    def modificar_x(self, x):
        self.x = x

    def modificar_y(self, y):
        self.y = y

    def modificar_z(self, z):
        self.z = z

def f(x, v):
    pid = os.getpid()
    match x:
        case 0:
            v.modificar_x(pid)
        case 1:
            v.modificar_y(pid)
        case 2:
            v.modificar_z(pid)


class Manager(multiprocessing.managers.BaseManager):
    pass

Manager.register("Vector", Vector)

with Manager() as mgr:
    v = mgr.Vector(0, 0, 0)

    procesos = [multiprocessing.Process(target=f, args=(i, v)) for i in range(3)]

    for p in procesos:
        p.start()

    for p in procesos:
        p.join()

    print(v.ver())



(22304, 22306, 22311)


Para obtener la instancia y poder usarla fuera del `Manager` hay que emplear el método `._getvalue()`:

In [99]:
with Manager() as mgr:
    v = mgr.Vector(0, 0, 0)

    procesos = [multiprocessing.Process(target=f, args=(i, v)) for i in range(3)]

    for p in procesos:
        p.start()

    for p in procesos:
        p.join()

    print(v.ver())
    v_obj = v._getvalue()

print(v_obj.x)

(22491, 22493, 22496)
22491
